In [ ]:
#BASE IDEA

N_BLOCKS = 100

w3 = Web3(Web3.HTTPProvider("https://still-powerful-cherry.discover.quiknode.pro/d19f575699ccc200b61ee6d889113e34a1797e62/"))
latest_block_number = w3.eth.block_number
N_BLOCKS = latest_block_number - N_BLOCKS

def get_block(block_number):
    
    block = w3.eth.get_block(block_number, full_transactions=True)
    block = dict(block)
    return(block)
blocks = []
for n in range(N_BLOCKS, latest_block_number):
    
    blocks.append(get_block(n))
print(blocks[0])


In [ ]:
import json
import sqlite3
from web3 import Web3
from web3.datastructures import AttributeDict
from hexbytes import HexBytes

N_BLOCKS = 20

w3 = Web3(Web3.HTTPProvider("https://still-powerful-cherry.discover.quiknode.pro/d19f575699ccc200b61ee6d889113e34a1797e62/"))
latest_block_number = w3.eth.block_number
N_BLOCKS = latest_block_number - N_BLOCKS

# Set up the SQLite database
conn = sqlite3.connect('./db/block1.db')
c = conn.cursor()

# Create tables
c.execute('''
CREATE TABLE IF NOT EXISTS blocks
(id TEXT PRIMARY KEY, baseFeePerGas TEXT, difficulty TEXT, extraData TEXT, gasLimit TEXT, 
gasUsed TEXT, hash TEXT, logsBloom TEXT, miner TEXT, mixHash TEXT, nonce TEXT, 
number TEXT, parentHash TEXT, receiptsRoot TEXT, sha3Uncles TEXT, size TEXT, 
stateRoot TEXT, timestamp TEXT, totalDifficulty TEXT, transactionsRoot TEXT, 
uncles TEXT, withdrawals TEXT, withdrawalsRoot TEXT)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS transactions
(id INTEGER PRIMARY KEY, block_id TEXT, tx_index INTEGER, transaction_data TEXT)
''')

def get_block(block_number):
    
    block = w3.eth.get_block(block_number, full_transactions=True)
    block = dict(block)
    
    # Insert block data into the blocks table
    c.execute('''
    INSERT OR IGNORE INTO blocks VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (str(block_number), str(block['baseFeePerGas']), str(block['difficulty']), str(block['extraData']), str(block['gasLimit']), str(block['gasUsed']), str(block['hash'].hex()), 
          str(block['logsBloom'].hex()), str(block['miner']), str(block['mixHash'].hex()), str(block['nonce'].hex()), str(block['number']), str(block['parentHash'].hex()), str(block['receiptsRoot'].hex()), 
          str(block['sha3Uncles'].hex()), str(block['size']), str(block['stateRoot'].hex()), str(block['timestamp']), str(block['totalDifficulty']), str(block['transactionsRoot'].hex), 
          str(block['uncles']), str(block['withdrawals']), str(block['withdrawalsRoot'])))
    
    # Insert each transaction into the transactions table
    for tx_index, tx in enumerate(block['transactions']):
        tx = {k: v.hex() if isinstance(v, HexBytes) else str(v) for k, v in tx.items()}
        c.execute('''
        INSERT OR IGNORE INTO transactions VALUES (?, ?, ?, ?)
        ''', (None, str(block_number), tx_index, str(tx)))
    return block
blocks = []
for n in range(N_BLOCKS, latest_block_number):
    blocks.append(get_block(n))

# Commit the changes and close the connection
conn.commit()
conn.close()

print(blocks)


In [14]:

def get_block_by_hash(block_hash):
    
    conn = sqlite3.connect('./db/block1.db')
    c = conn.cursor()
    c.execute('''
    SELECT * FROM blocks WHERE hash = ?
    ''', (block_hash,))
    result = c.fetchone()
    conn.close()

    return result

def get_transaction(transaction_hash):
    conn = sqlite3.connect('./db/block1.db')
    c = conn.cursor()
    c.execute('''
    SELECT * FROM transactions WHERE transaction_data LIKE ?
    ''', ('%{}%'.format(transaction_hash),))
    result = c.fetchone()
    conn.close()

    return result

#block_hash = "0x3be81d14b10e19508e4dba2df7ee4a194ba9d9b85559e81600bd9a5ac53a94e2"
transaction_hash = "0x977b0f69a4d1f0209ca9c64e46a256e12142fb3802bacfb559106b7a31ba3c30"

#print(get_block(block_hash))
print(get_transaction(transaction_hash))


(1, '17269813', 0, "{'blockHash': '0xfb2afa8a9c6f2f53b348798884c8628becca1af4151b3e55cb16aedecf322d93', 'blockNumber': '17269813', 'from': '0x85B4ad5f4F563e549a33D5b0b23c1091AA03d2Ac', 'gas': '500000', 'gasPrice': '684306484304', 'maxFeePerGas': '0xa0d6823572', 'maxPriorityFeePerGas': '0x933e89034e', 'hash': '0x977b0f69a4d1f0209ca9c64e46a256e12142fb3802bacfb559106b7a31ba3c30', 'input': '0x8581fb45000000000000000000000000ecf678bf1850ac206f42c9e959f72a0ab0e86f7700000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000ffffffffffffffffffffffffffffffffffffffff4557ba4e11ab53ef4fc67988000000000000000000000000000000000000000000000000323db83f02c672f90000000000000000000000007702f834314e4cbaf9bd8e7f0fae8275e744d9660000000000000000000000000000000000000000000000000000000000000020000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'nonce': '43299', 'to': '0xBD4fECd9337d22514fDD6d7fbeb9155cF8D790B1', 'transactionInd

In [15]:
import requests
import json

block_hashes = ["0xfb2afa8a9c6f2f53b348798884c8628becca1af4151b3e55cb16aedecf322d93", "0x1ad85adf0fe80538014b4b677e884b77edb5fdb73966c40b47aef4cde8e62354"]

tx_hashes = [
    "0x977b0f69a4d1f0209ca9c64e46a256e12142fb3802bacfb559106b7a31ba3c30",
    "0x5654fbd35ac0b419a9a98f934a61b3ce070b82267e0bbf8013fa79d67da31fb7",
    "0x77a53689a793fc194869f9877b06288b62b913fd4e05737da0009acc630e0de1",
]

for block_hash in block_hashes:
    response = requests.get(f"http://localhost:5000/block/{block_hash}")
    print(f"For block hash: {block_hash}")
    print("Response status code:", response.status_code)
    print("Response content:", json.dumps(response.json(), indent=4))

for tx_hash in tx_hashes:
    response = requests.get(f"http://localhost:5000/transaction/{tx_hash}")
    print(f"For transaction hash: {tx_hash}")
    print("Response status code:", response.status_code)
    print("Response content:", json.dumps(response.json(), indent=4))


For block hash: 0xfb2afa8a9c6f2f53b348798884c8628becca1af4151b3e55cb16aedecf322d93
Response status code: 200
Response content: [
    "17269813",
    "0xc154f4902",
    "0",
    "b'Illuminate Dmocratize Dstribute'",
    "30000000",
    "12039481",
    "0xfb2afa8a9c6f2f53b348798884c8628becca1af4151b3e55cb16aedecf322d93",
    "0x3021001e5ba00157c89a30edab8244e3053a0155441004c80a4b14d22523071c450002e8902081a0d0905c21424d0d00f6194881c908a810bb10cd4d792e1f140800701824400e380f45c64aca9581e018510ab4b6c34dc3c6b1104cc3ecf00212309c049f1e20139fac1849219c2c61d13c91240098072d900089d6820e443638c5e54204a0987894db5872038aa21b253cb4018980425a055040c108945114aa7802550884b121601270f491580c8546d5492c60c6e88a50618c340e28b04a0d3804430508285a6920481024fedc32464a824b17a22912001c194b10772072f1b2380090a0a38587641580ca697c19901a9012618a266a9c28b213c88c2410",
    "0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5",
    "0x1a0cc3a911ec4ccc611502766e79b43f6bb267954a4a4584172bd66cd2d80f7c",
    "0x0000000000000000",
    "17

In [7]:
#Debug
import sqlite3

# Re-open the SQLite database
conn = sqlite3.connect('./db/block1.db')
c = conn.cursor()

# Fetch the first 5 rows from the blocks table
c.execute('''
SELECT * FROM blocks LIMIT 5
''')
print("First 5 rows from the blocks table:")
for row in c.fetchall():
    print(row)

# Fetch the first 5 rows from the transactions table
c.execute('''
SELECT * FROM transactions LIMIT 5
''')
print("\nFirst 5 rows from the transactions table:")
for row in c.fetchall():
    print(row)

# Close the connection
conn.close()
#10 12 19

First 5 rows from the blocks table:
('17269813', '0xc154f4902', '0', "b'Illuminate Dmocratize Dstribute'", '30000000', '12039481', '0xfb2afa8a9c6f2f53b348798884c8628becca1af4151b3e55cb16aedecf322d93', '0x3021001e5ba00157c89a30edab8244e3053a0155441004c80a4b14d22523071c450002e8902081a0d0905c21424d0d00f6194881c908a810bb10cd4d792e1f140800701824400e380f45c64aca9581e018510ab4b6c34dc3c6b1104cc3ecf00212309c049f1e20139fac1849219c2c61d13c91240098072d900089d6820e443638c5e54204a0987894db5872038aa21b253cb4018980425a055040c108945114aa7802550884b121601270f491580c8546d5492c60c6e88a50618c340e28b04a0d3804430508285a6920481024fedc32464a824b17a22912001c194b10772072f1b2380090a0a38587641580ca697c19901a9012618a266a9c28b213c88c2410', '0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5', '0x1a0cc3a911ec4ccc611502766e79b43f6bb267954a4a4584172bd66cd2d80f7c', '0x0000000000000000', '17269813', '0x6c5ffa9a34d3bcca64ccc77aa9fc08bb1ccfca7709188991d77935541c538f9e', '0x6c904f63b40edaba47f1a9b1aff250800c2b631d1f92e7ff80c4aa9b4e